In [2]:
"""We didnt have time to correct minor differences between the kaggle dataset and recent dataset, so we  
were not able to use the same notebook for the analysis of both sets. 
As such, there is a lot of code repetition between this notebook and Analyse_Data_Kaggle.ipynb"""

import pandas as pd

In [3]:
"""Identify the categories with least expression and change its category to 'Other', in each group of country, 
category, year and month"""

def categories_to_merge(df):
    # rank each group and return the categories with the least expression
    df = df.rank(axis=0, pct=True).reset_index()
    return list(df[df["aggregation"] < 0.3]["category"].unique())

def assign_category_other(row,list_categories):
    # change the category to 'Other' if the original category is in the list
    if str(row["category"]) in list_categories:
        return "Other" 
    else:
        return row["category"]

def build_table(df,column,how):
    df_aux = df.copy()
    
    # group by country, category, year and month. 
    # the argument 'column' gives us the column we want to aggregate.
    # the argument 'how' gives us the type of aggregation (sum, count...)
    output = df_aux.groupby(["country","category"]).agg({column:how})
    output.rename(columns = {column:"aggregation"}, inplace = True)

    # analyse the minoritary categories and change those categories to 'Other'
    list_categories_to_merge = categories_to_merge(output.copy())
    print("Minoritary categories: " + ",".join(list_categories_to_merge))
    output.reset_index(inplace=True)
    df_aux["category"] = df_aux.apply(assign_category_other,axis=1,args=(list_categories_to_merge,))

    # group again by country, category, year and month
    output = df_aux.groupby(["country","category"]).agg({column:how})
    output.rename(columns = {column:"aggregation"}, inplace = True)
    
    # pivot the table so we have a column per category
    output = output.pivot_table(index=['country'], columns='category')
    output.to_csv(column+"_"+how+"_recent_raw.csv")
    
    return output




In [4]:
# import the csv

videos_recent = pd.read_csv("Recent_videos_merged.csv")
videos_recent.head()

,video_id,title,publish_time,channelId,channel_title,category_id,trending_date,views,likes,comment_count,comments_disabled,ratings_disabled,category,country
0,41W_Dx_Ajj4,"Nuno Ribeiro, Calema, Mariza - Maria Joana (Of...",2023-04-14T11:00:07Z,UCP6u9OBUd4M6U4yzSuv4UNw,Klasszik,10.0,23.27.04,1814964.0,38655.0,1224.0,False,False,Music,PT
1,pZCvOWR3T_I,Cama de Pregos #shorts,2023-04-12T17:08:52Z,UCNrthyyJVjrJUpePrl6lW_g,Neagle,24.0,23.27.04,9479027.0,845512.0,1885.0,False,False,Entertainment,PT
2,GGBy50KtvRk,Tipping A Waitress A Car,2023-04-22T20:00:00Z,UCX6OQ3DkcsbYNE6H8uQQuVA,MrBeast,24.0,23.27.04,31736198.0,2394370.0,9664.0,False,False,Entertainment,PT
3,M3L_zsHjomo,🖤❤️ Respect Jeremie Frimpong 🫶,2023-04-15T15:40:18Z,UCSMZmPVql528Cph9WPvt0GA,Bayer 04 Leverkusen,17.0,23.27.04,14182873.0,623615.0,4242.0,False,False,Sports,PT
4,cimLkAVuuv0,Usei a Roupa Sauna por 1 dia #shorts,2023-04-22T22:32:51Z,UCNrthyyJVjrJUpePrl6lW_g,Neagle,24.0,23.27.04,2449073.0,209871.0,740.0,False,False,Entertainment,PT


In [5]:
# Build the table for category (agg = count)

build_table(videos_recent,"category","count")

Minoritary categories: Autos & Vehicles,Education,Film & Animation,Howto & Style,News & Politics,Travel & Events,Pets & Animals,Science & Technology


aggregation                                                       
category      Comedy Entertainment Gaming Music Other People & Blogs Sports
country                                                                    
BR                 3            44     50    33    12             21     37
ES                 8            44     28    39    28             10     33
KE                13            47      3    33    17             13     34
NP                 7            42      3    10    20             32      6
PT                 5            43      6    21    10             10     12

In [6]:
# Build the table for views (agg = sum)
build_table(videos_recent,"views","sum")

Minoritary categories: Autos & Vehicles,Comedy,Education,Film & Animation,Howto & Style,News & Politics,Science & Technology,Travel & Events


aggregation                                                         
category Entertainment       Gaming        Music        Other People & Blogs   
country                                                                        
BR        5.542582e+07   39149650.0  202349381.0    6687989.0   1.052856e+07  \
ES        1.548236e+08   18033599.0  119725179.0  110904262.0   5.427310e+07   
KE        1.088767e+09  166424041.0  141205484.0  474009786.0   5.483291e+08   
NP        8.917953e+08  163069891.0  154944218.0  492880322.0   1.070707e+09   
PT        1.306704e+09   60127584.0  423372808.0  357605357.0   3.677031e+08   

                                      
category Pets & Animals       Sports  
country                               
BR                  NaN   15556426.0  
ES                  NaN   56825419.0  
KE          118145538.0  166106683.0  
NP                  NaN   79071138.0  
PT                  NaN  120487065.0

In [7]:
# Build the table for comment_count (agg = sum)
build_table(videos_recent,"comment_count","sum")

Minoritary categories: Autos & Vehicles,Comedy,Education,Film & Animation,Howto & Style,Science & Technology,Travel & Events,Pets & Animals


aggregation                                                 
category Entertainment    Gaming      Music News & Politics    Other   
country                                                                
BR             88338.0  105672.0   636372.0          6230.0  11642.0  \
ES            125347.0   54066.0   558280.0         28889.0  27897.0   
KE            228071.0   17862.0   526206.0         12846.0  50995.0   
NP            165546.0   89734.0   192422.0          4838.0  35489.0   
PT            343695.0   13933.0  3043299.0         14134.0  37934.0   

                                  
category People & Blogs   Sports  
country                           
BR              65720.0  38468.0  
ES              42976.0  35990.0  
KE              67868.0  56668.0  
NP             217631.0   5541.0  
PT              39392.0  15850.0

In [8]:
# Build the table for likes (agg = sum)
build_table(videos_recent,"likes","sum")

Minoritary categories: Autos & Vehicles,Comedy,Education,Film & Animation,Howto & Style,News & Politics,Science & Technology,Travel & Events


aggregation                                                     
category Entertainment     Gaming       Music       Other People & Blogs   
country                                                                    
BR           2604354.0  1965522.0   8170505.0    363531.0       725682.0  \
ES           7585767.0  1247021.0   6490866.0   3625048.0      1399484.0   
KE          30739625.0  4975640.0   7575158.0  11323266.0     12930063.0   
NP          27750970.0  4586259.0   5418848.0  11020606.0     31870499.0   
PT          42279177.0  3212060.0  21858452.0   9994550.0     11119120.0   

                                    
category Pets & Animals     Sports  
country                             
BR                  NaN   759794.0  
ES                  NaN  1414670.0  
KE            1491573.0  3762124.0  
NP                  NaN  2413675.0  
PT                  NaN  4882838.0